In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
#from sklearn.model_selection import train_test_split
import os
import keras_tuner as kt
import cv2

In [ ]:
'''
Michael Dawson-Haggerty

abb.py: contains classes and support functions which interact with an ABB Robot running our software stack (RAPID code module SERVER)


For functions which require targets (XYZ positions with quaternion orientation),
targets can be passed as [[XYZ], [Quats]] OR [XYZ, Quats]

'''

import socket
import json 
import time
import inspect
from threading import Thread
from collections import deque
import logging

log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())
    
class Robot:
    def __init__(self, 
                 ip          = '192.168.125.1', 
                 port_motion = 5000,
                 port_logger = 5001):

        self.delay   = .08

        self.connect_motion((ip, port_motion))
        #log_thread = Thread(target = self.get_net, 
        #                    args   = ((ip, port_logger))).start()
        
        self.set_units('millimeters', 'degrees')
        self.set_tool()
        self.set_workobject()
        self.set_speed()
        self.set_zone()

    def connect_motion(self, remote):        
        log.info('Attempting to connect to robot motion server at %s', str(remote))
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.settimeout(2.5)
        self.sock.connect(remote)
        self.sock.settimeout(None)
        log.info('Connected to robot motion server at %s', str(remote))

    def connect_logger(self, remote, maxlen=None):
        self.pose   = deque(maxlen=maxlen)
        self.joints = deque(maxlen=maxlen)
        
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect(remote)
        s.setblocking(1)
        try:
            while True:
                data = map(float, s.recv(4096).split())
                if   int(data[1]) == 0: 
                    self.pose.append([data[2:5], data[5:]])
                #elif int(data[1]) == 1: self.joints.append([a[2:5], a[5:]])
        finally:
            s.shutdown(socket.SHUT_RDWR)

    def set_units(self, linear, angular):
        units_l = {'millimeters': 1.0,
                   'meters'     : 1000.0,
                   'inches'     : 25.4}
        units_a = {'degrees' : 1.0,
                   'radians' : 57.2957795}
        self.scale_linear = units_l[linear]
        self.scale_angle  = units_a[angular]

    def set_cartesian(self, pose):
        '''
        Executes a move immediately from the current pose,
        to 'pose', with units of millimeters.
        '''
        msg  = "01 " + self.format_pose(pose)   
        return self.send(msg)

    def set_joints(self, joints):
        '''
        Executes a move immediately, from current joint angles,
        to 'joints', in degrees. 
        '''
        if len(joints) != 6: return False
        msg = "02 "
        for joint in joints: msg += format(joint*self.scale_angle, "+08.2f") + " " 
        msg += "#" 
        return self.send(msg)

    def get_cartesian(self):
        '''
        Returns the current pose of the robot, in millimeters
        '''
        msg = "03 #"
        data = self.send(msg).split()
        r = [float(s) for s in data]
        return [r[2:5], r[5:9]]

    def get_joints(self):
        '''
        Returns the current angles of the robots joints, in degrees. 
        '''
        msg = "04 #"
        data = self.send(msg).split()
        return [float(s) / self.scale_angle for s in data[2:8]]

    def get_external_axis(self):
        '''
        If you have an external axis connected to your robot controller
        (such as a FlexLifter 600, google it), this returns the joint angles
        '''
        msg = "05 #"
        data = self.send(msg).split()
        return [float(s) for s in data[2:8]]
       
    def get_robotinfo(self):
        '''
        Returns a robot- unique string, with things such as the
        robot's model number. 
        Example output from and IRB 2400:
        ['24-53243', 'ROBOTWARE_5.12.1021.01', '2400/16 Type B']
        '''
        msg = "98 #"
        data = str(self.send(msg))[5:].split('*')
        log.debug('get_robotinfo result: %s', str(data))
        return data

    def set_tool(self, tool=[[0,0,0], [1,0,0,0]]):
        '''
        Sets the tool centerpoint (TCP) of the robot. 
        When you command a cartesian move, 
        it aligns the TCP frame with the requested frame.
        
        Offsets are from tool0, which is defined at the intersection of the
        tool flange center axis and the flange face.
        '''
        msg       = "06 " + self.format_pose(tool)    
        self.send(msg)
        self.tool = tool

    def load_json_tool(self, file_obj):
        if file_obj.__class__.__name__ == 'str':
            file_obj = open(filename, 'rb');
        tool = check_coordinates(json.load(file_obj))
        self.set_tool(tool)
        
    def get_tool(self): 
        log.debug('get_tool returning: %s', str(self.tool))
        return self.tool

    def set_workobject(self, work_obj=[[0,0,0],[1,0,0,0]]):
        '''
        The workobject is a local coordinate frame you can define on the robot,
        then subsequent cartesian moves will be in this coordinate frame. 
        '''
        msg = "07 " + self.format_pose(work_obj)   
        self.send(msg)

    def set_speed(self, speed=[100,50,50,50]):
        '''
        speed: [robot TCP linear speed (mm/s), TCP orientation speed (deg/s),
                external axis linear, external axis orientation]
        '''

        if len(speed) != 4: return False
        msg = "08 " 
        msg += format(speed[0], "+08.1f") + " " 
        msg += format(speed[1], "+08.2f") + " "  
        msg += format(speed[2], "+08.1f") + " " 
        msg += format(speed[3], "+08.2f") + " #"     
        self.send(msg)

    def set_zone(self, 
                 zone_key     = 'z1', 
                 point_motion = False, 
                 manual_zone  = []):
        zone_dict = {'z0'  : [.3,.3,.03], 
                    'z1'  : [1,1,.1], 
                    'z5'  : [5,8,.8], 
                    'z10' : [10,15,1.5], 
                    'z15' : [15,23,2.3], 
                    'z20' : [20,30,3], 
                    'z30' : [30,45,4.5], 
                    'z50' : [50,75,7.5], 
                    'z100': [100,150,15], 
                    'z200': [200,300,30]}
        '''
        Sets the motion zone of the robot. This can also be thought of as
        the flyby zone, AKA if the robot is going from point A -> B -> C,
        how close do we have to pass by B to get to C
        
        zone_key: uses values from RAPID handbook (stored here in zone_dict)
        with keys 'z*', you should probably use these

        point_motion: go to point exactly, and stop briefly before moving on

        manual_zone = [pzone_tcp, pzone_ori, zone_ori]
        pzone_tcp: mm, radius from goal where robot tool centerpoint 
                   is not rigidly constrained
        pzone_ori: mm, radius from goal where robot tool orientation 
                   is not rigidly constrained
        zone_ori: degrees, zone size for the tool reorientation
        '''

        if point_motion: 
            zone = [0,0,0]
        elif len(manual_zone) == 3: 
            zone = manual_zone
        elif zone_key in zone_dict.keys(): 
            zone = zone_dict[zone_key]
        else: return False
        
        msg = "09 " 
        msg += str(int(point_motion)) + " "
        msg += format(zone[0], "+08.4f") + " " 
        msg += format(zone[1], "+08.4f") + " " 
        msg += format(zone[2], "+08.4f") + " #" 
        self.send(msg)

    def buffer_add(self, pose):
        '''
        Appends single pose to the remote buffer
        Move will execute at current speed (which you can change between buffer_add calls)
        '''
        msg = "30 " + self.format_pose(pose) 
        self.send(msg)

    def buffer_set(self, pose_list):
        '''
        Adds every pose in pose_list to the remote buffer
        '''
        self.clear_buffer()
        for pose in pose_list: 
            self.buffer_add(pose)
        if self.buffer_len() == len(pose_list):
            log.debug('Successfully added %i poses to remote buffer', 
                      len(pose_list))
            return True
        else:
            log.warn('Failed to add poses to remote buffer!')
            self.clear_buffer()
            return False

    def clear_buffer(self):
        msg = "31 #"
        data = self.send(msg)
        if self.buffer_len() != 0:
            log.warn('clear_buffer failed! buffer_len: %i', self.buffer_len())
            raise NameError('clear_buffer failed!')
        return data

    def buffer_len(self):
        '''
        Returns the length (number of poses stored) of the remote buffer
        '''
        msg = "32 #"
        data = self.send(msg).split()
        return int(float(data[2]))

    def buffer_execute(self):
        '''
        Immediately execute linear moves to every pose in the remote buffer.
        '''
        msg = "33 #"
        return self.send(msg)

    def set_external_axis(self, axis_unscaled=[-550,0,0,0,0,0]):
        if len(axis_values) != 6: return False
        msg = "34 "
        for axis in axis_values:
            msg += format(axis, "+08.2f") + " " 
        msg += "#"   
        return self.send(msg)

    def move_circular(self, pose_onarc, pose_end):
        '''
        Executes a movement in a circular path from current position, 
        through pose_onarc, to pose_end
        '''
        msg_0 = "35 " + self.format_pose(pose_onarc)  
        msg_1 = "36 " + self.format_pose(pose_end)

        data = self.send(msg_0).split()
        if data[1] != '1': 
            log.warn('move_circular incorrect response, bailing!')
            return False
        return self.send(msg_1)

    def set_dio(self, value, id=0):
        '''
        A function to set a physical DIO line on the robot.
        For this to work you're going to need to edit the RAPID function
        and fill in the DIO you want this to switch. 
        '''
        msg = '97 ' + str(int(bool(value))) + ' #'
        return 
        #return self.send(msg)
        
    def send(self, message, wait_for_response=True):
        '''
        Send a formatted message to the robot socket.
        if wait_for_response, we wait for the response and return it
        '''
        caller = inspect.stack()[1][3]
        log.debug('%-14s sending: %s', caller, message)
        self.sock.send(bytes(message, encoding='utf-8'))
        time.sleep(self.delay)
        if not wait_for_response: return
        data = self.sock.recv(4096)
        log.debug('%-14s recieved: %s', caller, data)
        return data
        
    def format_pose(self, pose):
        pose = check_coordinates(pose)
        msg  = ''
        for cartesian in pose[0]:
            msg += format(cartesian * self.scale_linear,  "+08.1f") + " " 
        for quaternion in pose[1]:
            msg += format(quaternion, "+08.5f") + " " 
        msg += "#" 
        return msg       
        
    def close(self):
        self.send("99 #", False)
        self.sock.shutdown(socket.SHUT_RDWR)
        self.sock.close()
        log.info('Disconnected from ABB robot.')

    def __enter__(self):
        return self
        
    def __exit__(self, type, value, traceback):
        self.close()

def check_coordinates(coordinates):
    if ((len(coordinates) == 2) and
        (len(coordinates[0]) == 3) and 
        (len(coordinates[1]) == 4)): 
        return coordinates
    elif (len(coordinates) == 7):
        return [coordinates[0:3], coordinates[3:7]]
    log.warn('Recieved malformed coordinate: %s', str(coordinates))
    raise NameError('Malformed coordinate!')

if __name__ == '__main__':
    formatter = logging.Formatter("[%(asctime)s] %(levelname)-7s (%(filename)s:%(lineno)3s) %(message)s", "%Y-%m-%d %H:%M:%S")
    handler_stream = logging.StreamHandler()
    handler_stream.setFormatter(formatter)
    handler_stream.setLevel(logging.DEBUG)
    log = logging.getLogger('abb')
    log.setLevel(logging.DEBUG)
    log.addHandler(handler_stream)
    


# ROBOT DOCS

Quick and Dirty Methods for ABB Communication Module

METHODS:

set_units(linear, angular) : 
	Allows the user to set the units of the input for the robot.
	
	linear (str) : Choice of millimeters, meters, or inches
	angular (str) : Choice of degrees or radians


set_cartesian(pose):
	Executes a linear move from the current position to the position indicated by pose.

	pose (cartesian position) : [[X, Y, Z], [q1,q2,q3,q4]] : the new position the robot is moving too.
		X, Y, and Z are the unit locations within the workObject and q1-q4 describe the 		orientation of the tool

set_joints(joints):
	directly sets the joint positions of the robot.

	joints (array) : [1,2,3,4,5,6] : an array with 6 positions where joints[0] represents the degree value of 	joint 1 (the base), and joints[5] represents the degree value of joint 6 (the tool face).

get_cartesian():
	returns an array containing the current position and orientation of the tool within the work object.

	RETURNS: a “cartesian position” (see “set_cartesian”) containing the current position of the tool.

get_joints():
	Returns an array containing the current degree values of the joints.

	RETURNS: an array with 6 positions, relating to the degree rotation of each of the 6 axis’

get_external_axis():
	NOT NEEDED FOR OUR PROJECT. Returns the position of an external axis.

get_robotinfo():
	Returns a string containing information such as the robot’s model number, RobotWare versions, and 	other similar info.

set_tool(tool):
	Allows the user to define a tool center-point. Defining tools allows for more precise movement.

	tool (TCP data): [[X, Y, Z], [q1,q2,q3,q4]] : Very similar to a cartesian position. Essentially a position 	from the frame of reference of the tool plate, with X Y Z and orientation positions saved.

	NOTE: Though I do not see it here, tool data also usually includes its weight in kilograms and 	information to indicate the location of the tool’s center of mass. This allows the ABB software to take 	these factors into consideration when making moves while keeping set speeds.

load_json_tool(file_obj):
	(UNSURE?) Looks to be another way to load tool data using a JSON format.

get_tool():
	Returns the data for the currently set tool.

	RETURNS: TCP data of currently active tool. (See “set_tool”)

set_workobject(work_obj):
	Allows the user to set the current workObject data for the ABB Arm.
	WorkObjects are like frames of reference, they allow the user to define a specific coordinate systems 	for a surface or object that is independent of rotation or actual position in the world.

	work_obj (WorkOBJData) : [[X, Y, Z], [q1,q2,q3,q4]] : Very similar to cartesian position. X, Y, and Z 	refer to the position in space of the origin of the work object. Q1-q4 refer to the orientation relative to 	the world object

set_speed(speed):
	Allows the user to set the speed of the robot.

	speed (array) : [LSpeed, RSpeed, ExtLSpeed, ExtRSpeed] : Speed variables corresponding to linear 	tool speed (mm/s), rotational tool speed (degrees/s), external axis speed (mm/s), external axis 	rotational speed (degrees/s)

set_zone(zone_key, point_motion, manual_zone):
	Allows the user to set the zone of the robot. Zone refers to the amount of tolerance, in terms of 	distance to a point, the robot has when considering its moves. Better explained in the Functions and 	DataTypes PDF.

	zone_key (str) : Refers to a set of preconfigured zone data values. (z0, z1, z5, z15, z20, z30, z50, 	z100, z200)

	point_motion (bool) : Refers specifically to “fine” point movements. I.E. Robot moves to the exact 	point position of a variable before moving to the next one.

	manual_zone (array) : [pzone_tcp, pzone_ori, zone_ori] :
        		pzone_tcp: mm, radius from goal where robot tool centerpoint 
                   		is not rigidly constrained
        		pzone_ori: mm, radius from goal where robot tool orientation 
                   		is not rigidly constrained
        		zone_ori: degrees, zone size for the tool reorientation

buffer_add(pose):
	Appends single pose to the remote buffer. Move will execute at current speed (which you can change 	between buffer_add calls)


buffer_set(pose_list): 
	Add every pose in pose_list to the remote buffer.

clear_buffer():
	Clears remote buffer

buffer_len():
	Returns the length of the remote buffer.

buffer_execute():
	Executes linear moves to every pose in the remote buffer.

set_external_axis(axis_unscaled):
	NOT NEEDED FOR OUR PROJECT. Sets an external axis for the robot.

move_circular(pose_onarc, pose_end):
	Allows the user to move the tool in a circular arc from the current position, to the peak of the arc at 	‘pose_onarc’, to the end of the arc at “pose_end’.

	pose_onarc (cartesian position) : the position at the peak of an arc.
	
	pose_end (cartesian position) : the position at the end of an arc.

	NOTE: This uses the MoveC RobotWare command. In my experience this command is particular 	finicky. I requires minimum and maximum arc angles for it to run properly. Use at your own risk.

set_dio(value, id):
	Allows the user to control the robot’s digital GPIO inputs.

	value (bool) : set IO to True or False
	
	id (int) : The specific GPIO? This doesn’t appear to be used in the abb.py program??? Have to double 	check this if we want to use GPIO pins

send(message, wait_for_response=True):
	This module sends data from the python program to the PC interface of the robot.
	
	message (str) : String message containing data for executing commands.
	wait_for_response (bool) : (ASSUMED) Whether or not we should wait for a response from the robot.

	NOTE: If we want to use Python3, this module will have to be updated.

format_pose(pose):
	(UNSURE?) My guess is that it converts a cartesian point (pose) into different units if necessary. 	Possibly converts info into string data? Not quite sure.

close():
	Disconnects user from robot.

check_coordinates(coordinates):
	confirms that coordinates are of the correct cartesian formatting.

In [ ]:
import cv2
from PIL import Image

def start_cap():
    cv2.destroyAllWindows()
    imagenum = 1
    capturedImg = None
    cam = cv2.VideoCapture(1)

    while True:
        ret_val, capturedImg = cam.read()
        print("ret_val:", ret_val)
        print("capturedImg shape:", capturedImg.shape if capturedImg is not None else None)

        cv2.imshow('my webcam', capturedImg)
        if cv2.waitKey(1) == 0:
            break
        if cv2.waitKey(1) == 13:
            Image.fromarray(capturedImg).save('imgcap' + str(imagenum) + '.jpg')
            imagenum += 1

    cam.release()
    cv2.destroyAllWindows()

start_cap()


In [ ]:
#connect 
R = Robot(ip='192.168.125.1')

In [ ]:
#R.set_joints([0,0,0,0,0,90]) example of joint positions
#R.set_cartesian([[400.0, 20.0, 600.0], [0.5, 0.5, 0.5, 0.5]]) example of xyz movement along with pose





In [ ]:
#close
R.set_joints([0,0,0,0,0,0])
R.close()

In [2]:
import cv2

# Create a VideoCapture object for the default camera (index 0)
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    # Read a single frame from the camera
    ret, frame = cap.read()

    # Check if the frame is read successfully
    if ret:
        # Display the captured frame
        cv2.imshow('Captured Image', frame)

        # Save the captured frame to a file (e.g., 'captured_image.jpg')
        cv2.imwrite('C:/Users/marco/Desktop/RobotProject/images/img1.jpg', frame)
        print("Image captured and saved.")

        # Wait for a key press and then close the OpenCV window
        cv2.waitKey(0)

    else:
        print("Error: Failed to capture frame.")

    # Release the camera and close the OpenCV window
    cap.release()
    cv2.destroyAllWindows()


Image captured and saved.


In [ ]:
import abb
import time
import datetime
from pyquaternion import Quaternion
import math
import random
import csv
import os
import cv2






class moveableObj:
    """A class for movable objects in our workspace. Tracks X,Y,Z position and rotation of objects, and the tool they are used to interact with"""
    def __init__(self, initLoc, initIsRot, initRot, toolData, toolMinMax, tool_command, initSpeed, radius, initApproachAngle, initHeight):

        #Tracks the current location of the cylinder in the world plane as [X, Y, Z] and its rotation relative to the quaternion of it's tool [q1,q2,q3,q4]
        self.cartLoc = initLoc

        #Tracks the degree and distance value given by the random number generator [deg, distance]
        self.polarLoc = [0.0,120.0]

        #Refers to the joint positions of the robot at the location of the object
        self.jointLoc = [0,0,0,0,0,0]

        #Bool value that determines if object should have random rotational values
        self.isRot = initIsRot

        #relative to the world, the degree value of rotation with respect to the XY plane
        self.rot = initRot

        #Establishes the tool used to interact with object
        self.tool = toolData

        #Stores the min and max polar coord value for the given tool
        self.toolMinMax = toolMinMax

        #Refers to the Set_Do command associated with the tool
        self.activate_tool = tool_command

        #Refers to the maximum lift speed of the object
        self.maxLiftSpeed = initSpeed

        #Refers to the loose radial bounding box around the centerpoint of the object
        self.radialEdge = radius

        #Refers to the angle at which the object should be approched
        self.approachAngle = initApproachAngle

        #Refers to the height of the object
        self.height = initHeight

        #Refers to the z modifier
        self.zMod = 0
        
    #adjusts the approach angle in the cartesian coords
    def getApproachQuat(self, modQuat):
        if self.approachAngle == 0:
                return modQuat
        else:
            q1 = Quaternion(modQuat)
            q2 = Quaternion(axis = [0.0,1.0,0.0], degrees = self.approachAngle)

            q3 = q1 * q2

            return list(q3)


class DatasetCollector:
    def __init__(self, total_photos, save_path):
        self.total_photos = total_photos
        self.save_path = save_path
        self.csv_file = os.path.join(save_path, 'dataset.csv')
        
        #print("Open CV")
        self.cap = cv2.VideoCapture(0)
        #print("Open CV Success")
        self.cap.set(3, 1920)
        self.cap.set(4, 1080)
        #check if output dir is there and create if not
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
            # Initialize CSV file
            with open(self.csv_file, 'w', newline='') as file:
                writer = csv.writer(file)
                #add data columns here
                writer.writerow(["Photo Name", "cyl.X", "cyl.Y", "cyl.Z", "cyl.q1", "cyl.q2", "cyl.q3", "cyl.q4",
        "cyl.r", "cyl.deg", "cyl.j1", "cyl.j2", "cyl.j3", "cyl.j4", "cyl.j5",
        "cyl.j6", "cyl.rot", "sqr.X", "sqr.Y", "sqr.Z", "sqr.q1", "sqr.q2",
        "sqr.q3", "sqr.q4", "sqr.r", "sqr.deg", "sqr.j1", "sqr.j2", "sqr.j3",
        "sqr.j4", "sqr.j5", "sqr.j6", "sqr.rot"])
        self.photo_count = len(os.listdir(self.save_path)) -1

    def capture_photo(self):
        self.photo_count += 1
        photo_name = self._generate_photo_name()
        photo_path = os.path.join(self.save_path, photo_name)


        ret, frame = self.cap.read()
        if ret:
            cv2.imwrite(photo_path, frame)

        return photo_name

    def update_csv(self, photo_name, objectList):
        row = []
        row.append(photo_name)

        for moveObj in objectList:
            row.append(moveObj.cartLoc[0][0])
            row.append(moveObj.cartLoc[0][1])
            row.append(moveObj.cartLoc[0][2])
            row.append(moveObj.cartLoc[1][0])
            row.append(moveObj.cartLoc[1][1])
            row.append(moveObj.cartLoc[1][2])
            row.append(moveObj.cartLoc[1][3])

            row.append(moveObj.polarLoc[0])
            row.append(moveObj.polarLoc[1])

            row.append(moveObj.jointLoc[0])
            row.append(moveObj.jointLoc[1])
            row.append(moveObj.jointLoc[2])
            row.append(moveObj.jointLoc[3])
            row.append(moveObj.jointLoc[4])
            row.append(moveObj.jointLoc[5])

            row.append(moveObj.rot)

        with open(self.csv_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)



    def _generate_photo_name(self):
        date_str = datetime.datetime.now().strftime("%d%H%M")
        return f"{date_str}_{self.photo_count}_{self.total_photos}.jpg"

    #Counts the amount of photos already taken. Returns the number of photos
    def count_init_data(self):
        files = os.listdir(self.save_path)
        self.photo_count = len(files) - 1
        return self.photo_count
        

    def close_class(self):
        self.cap.release()

class metricTracker:
    """A Class for stat tracking and time calculation"""
    def __init__(self, save_path, starting_iterations = 0):
        self.start_time = 0
        self.loop_time = 0
        self.total_time = 0
        self.iteration_count = starting_iterations
        self.run_count = 0
        self.fastest_time = float('inf')
        self.slowest_time = 0
        self.csv_filename = os.path.join(save_path, 'metric_log.csv')
        self.create_csv_file()

    def create_csv_file(self):
        if not os.path.exists(self.csv_filename):
            with open(self.csv_filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Iteration", "Previous Time", "Average Time", "Fastest Time", "Slowest Time", "Total Time"])

    def start_timer(self):
        self.start_time = time.time()

    def end_timer(self):
        end_time = time.time()
        self.loop_time = end_time - self.start_time
        self.update_metrics(self.loop_time)

    def update_metrics(self, loop_time):
        self.total_time += loop_time
        self.iteration_count += 1
        self.run_count += 1
        self.fastest_time = min(self.fastest_time, loop_time)
        self.slowest_time = max(self.slowest_time, loop_time)

    def get_average_time(self):
        return self.total_time / self.run_count if self.run_count else 0

    def estimate_remaining_time(self, total_iterations):
        average_time = self.get_average_time()
        remaining_iterations = total_iterations - self.iteration_count
        return remaining_iterations * average_time

    def output_to_csv(self):
        with open(self.csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(
                [self.run_count, self.loop_time, self.get_average_time(), self.fastest_time, self.slowest_time, self.total_time])

    def format_time(self, seconds):
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return f"{int(hours)}h {int(minutes)}m {int(seconds)}s"



## GLOBAL VARS ##

robIP = '192.168.125.1'

#TOOLS
#tGripper = [[1.1,11.5,139.6],[-0.70711,0.70711,.70711,.70711]]
'''
#SIMULATION VALUES
tGripper = [[1.1,11.5,139.6],[0.5,-0.5,-0.5,-0.5]]
tGripper_adv = [1,[[13.941,-25.105,74.149],[1,0,0,0]]]

tVac = [[1.58, -47.25, 24.06],[0.5,-0.5,-0.5,-0.5]]
tVac_adv = [1,[[13.941,-25.105,74.149],[1,0,0,0]]]

RBS MEASSURE
tGripper = [[-2.9189,28.3118,135.933],[0.5,-0.5,-0.5,-0.5]]
tGripper_adv = [.5,[[2,0,61.2],[1,0,0,0]]]

tVac = [[], [0.5,-0.5,-0.5,-0.5]]
tVac_adv = [.5,[[2,0,61.2],[1,0,0,0]]]
'''

tGripper = [[0,13.71,142.91],[0.5,-0.5,-0.5,-0.5]]
tGripper_adv = [.5,[[2,0,61.2],[1,0,0,0]]]

tVac = [[0,-42.72,35.37],[0.5,-0.5,-0.5,-0.5]]
#tVac = [[0,-46.72,35.37],[0.44528,-.54805,-.54936,-.44672]]
tVac_adv = [.5,[[2,0,61.2],[1,0,0,0]]]

#Manual Joint rotations
jHome = [0,45,45,0,-90,90]
jZero = [0,0,0,0,0,0]
jOffScreen = [90,0,0,0,0,0]
jOffScreenNeg = [-90,0,0,0,0,0]

#Speed Data

defaultSpeed = [550, 500, 5000, 1000]
moveSpeed = [800, 500, 5000, 1000]
squareLiftSpeed = [300,500,5000,1000]
'''

defaultSpeed = [500, 500, 5000, 1000]
moveSpeed = defaultSpeed
squareLiftSpeed = defaultSpeed
'''

defaultZone = [0,0,0]



#Starting Positions
cylinderStartPos = [[-337.2, 581.91, 70], [0.5, 0.0, 0.0, 0.866]]
squareStartPos = [[-199.9, 328.39, 0], [0.5,0.0,0.0,0.866]]
travelHeight = 120

#Program Parameters
picCount = 2000

image_dir = ''
csv_data_file = ''

#Spacing and Overlap Parameters
"""
General Sizing Notes:
Table = 48x49, 32 inches high (1219.2, 1244.6, 812.8 (mm))
Red Square = 7x7 (177.8 x 177.8)
cylinder whole = 2x6 (50.8 x 152.4)
non-top-hat-cylinder = 2x4 (50.8, 101.6)
top-hat-cylinder = 2x4 (50.8, 101.6)
arm = 27 high 31 wide

"""
armWidth = 200
#longMinDegreeSep = 11.2
#shortMinDegreeSep = 25

redSquareRadialEdge = 125.72
cylinderRadialEdge = 25.4

squareHeight = 0
cylinderHeight = 70

squareApproachAngle = 5
cylinderApproachAngle = 0

minSeperation = 40

maxMoveDistance = 500
minMoveDistance = 150

#Random number generator parameters
#Determined by tool size and reachable position relative to the table.
#ROTATIONAL VALUES: [degree values (-90 - 90), distance from center (tool dependant)]

#tGripper - range of 275.00
tGripperRange = [400, 600]

#tVac - range of 335.00
tVacRange = [300, 600]

#Degree Values
degMin = -90.0
degMax = 90.0




def initRobot():
    # Primary Robot Interface
    R = abb.Robot(ip=robIP)

    R.set_speed(defaultSpeed)
    R.set_zone("z0", point_motion=True)

    R.set_joints(jZero)
    R.set_tool_advanced(tGripper_adv)
    R.set_Do_Vacume(0)
    R.set_Do_Grip(0)

    return R

def camera_alignment(R):
    '''Set's Robot to proper postion for aligning camera jig'''
    R.set_joints(jHome)
    R.set_tool()
    R.set_tool_advanced()
    R.set_cartesian([[625, 0.0, 235], [0.5, 0.5, 0.5, 0.5]])

def quatPlaneRotate(quat = [1, 0, 0, 0], axisIn = [0.0, 0.0, 1.0], deg = 0):
    q1 = Quaternion(quat)
    q2 = Quaternion(axis = axisIn, degrees=deg)

    q3 = q1 * q2

    return list(q3)
"""
def getRelativePos(pos, mod):
    newPos = pos[:]
    newPos[0] = newPos[0] + mod[0]
    newPos[1] = newPos[1] + mod[1]
    newPos[2] = newPos[2] + mod[2]

    return newPos

def getRelativeJoint(joint, mod):
    newJoint = joint[:]

    newJoint[0] = newJoint[0] + mod[0]
    newJoint[1] = newJoint[1] + mod[1]
    newJoint[2] = newJoint[2] + mod[2]
    newJoint[3] = newJoint[3] + mod[3]
    newJoint[4] = newJoint[4] + mod[4]
    newJoint[5] = newJoint[5] + mod[5]

    return newJoint
"""

def getRelativeVal(val, mod):
    newVal = val[:]
    for i in range(len(newVal)):
        newVal[i] += mod[i]

    return newVal

def getAdjustedVal(val, adj):
    newVal = val[:]
    for i in range(len(newVal)):
        if adj[i] != 0:
            newVal[i] = adj[i]

    return newVal

def getBaseAngle(loc = [200,200,0]):
    '''Input [X,Y,Z] location and return the angle necessary to align axis 1 with the object'''
    angle = 0
    if(loc[1] != 0):
        angle = math.degrees(math.atan(abs(loc[1])/abs(loc[0])))
        if(loc[1] < 0):
            angle = angle*(-1)


    return angle

def getRelativeJHome(R1, mod = [0,0,0]):
    R1.set_joints(jHome)
    cuCoord = R1.get_cartesian()

    R1.set_cartesian([getAdjustedVal(cuCoord[0],mod), cuCoord[1]])
    print(str(R1.get_cartesian()))
    return R1.get_cartesian()

def get_min_degree_seperation(stillObject):
    """
    Calculates and returns the angle of seperation necessary to avoid unwanted collisions between two move objects

    Parameters:
    stillObject: The object we are avoiding

    Returns:
    A tuple of two values representing the angle range to avoid
    """
    if stillObject.polarLoc[0] != 0:
        angle = math.degrees(math.atan((armWidth/2) / stillObject.polarLoc[0]))
    else:
        angle = 0
    return (stillObject.polarLoc[1] - angle, stillObject.polarLoc[1] + angle)

def random_value_with_exclusions(min_val, max_val, exclusions, decimal_places=2):
    """
    Generates a random value between min_val and max_val, excluding specified ranges.
    Exclusions should be a list of tuples, each tuple representing a range to exclude.
    """
    while True:
        value = random.uniform(min_val, max_val)
        value = round(value, decimal_places)
        if not any(lower <= value <= upper for lower, upper in exclusions):
            return value


def calculate_distance(coords_list1, coords_list2):
    """
    Calculates the Euclidean distance between two points represented by two lists of X and Y coordinates.

    Parameters:
    coords_list1: A list containing the X and Y coordinates of the first point [X1, Y1].
    coords_list2: A list containing the X and Y coordinates of the second point [X2, Y2].

    Returns:
    The Euclidean distance between the two points.
    """

    if len(coords_list1) != 2 or len(coords_list2) != 2:
        raise ValueError("Each list must contain exactly two elements representing X and Y coordinates.")

    x_diff = coords_list1[0] - coords_list2[0]
    y_diff = coords_list1[1] - coords_list2[1]

    distance = math.sqrt(x_diff ** 2 + y_diff ** 2)
    return distance

def polar_to_cartesian(r, theta):
    """
    Converts polar coordinates to Cartesian coordinates.

    Parameters:
    r: Radius (distance from the origin).
    theta: Angle in degrees from the X-axis (-90 to 90 degrees).

    Returns:
    A tuple (x, y) representing Cartesian coordinates.
    """

    # Convert theta from degrees to radians
    theta_rad = math.radians(theta)

    # Calculate x and y using the polar to Cartesian conversion formulas
    x = r * math.cos(theta_rad)
    y = r * math.sin(theta_rad)

    return [x, y]



def executeObjectRotation(R1, rotationObject, deg = 0.0):
    '''Executes robot movements to rotate movable objects'''
    liftHeight = 25
    dropHeight = 10

    degreeMod = 0
    #move robot to rotation work area
    #R1.set_joints(getRelativeVal(jHome,[-120,0,0,0,0,0]))
    if (-55 < rotationObject.polarLoc[1] < -45):
        R1.set_joints(getRelativeVal(jHome,[-120,0,0,0,0,0]))
    elif (45 < rotationObject.polarLoc[1] < 55):
        R1.set_joints(getRelativeVal(jHome,[120,0,0,0,0,0]))
    
    if abs(R1.get_joints()[0]) > 90:
        currentLoc = R1.get_cartesian()

        workCords = polar_to_cartesian(400, R1.get_joints()[0])
        workCords.append(rotationObject.cartLoc[0][2])
        workQuat = currentLoc[1]
    else:
        workCords = polar_to_cartesian(400, rotationObject.polarLoc[1])
        workCords.append(rotationObject.cartLoc[0][2])
        workQuat = rotationObject.cartLoc[1]
    
    
    workspaceLoc = [workCords, workQuat]

    R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, liftHeight]), workspaceLoc[1]])

    #Place rot object down
    #R1.set_cartesian(workspaceLoc)
    #rotationObject.activate_tool(0)
    #R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, liftHeight]), workspaceLoc[1]])


    #Determine rotation direction
    rotationDirection = 0
    if(rotationObject.rot <= deg):
        #clockwise rotation
        rotationDirection = 1
    else:
        #counterclockwise rotation
        rotationDirection = -1


    while(rotationObject.rot != deg):
        #repeate this loop until rotation of object is correct
        #turn at maximum 90 degree turns
        rotAmount = 0
        if(rotationDirection*(deg - rotationObject.rot) >= 90):
            #current degree is more than 90 degrees away, turn 90
            rotAmount = 90*rotationDirection
        else:
            rotAmount = deg - rotationObject.rot

        rotQuat = quatPlaneRotate(workspaceLoc[1], [0.0,0.0,1.0], rotAmount)



        #do pickup
        #R1.set_cartesian(workspaceLoc)
        #rotationObject.activate_tool(1)
        #R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, liftHeight]), workspaceLoc[1]])

        #rotate
        R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, liftHeight]), rotQuat])

        #place back
        R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, dropHeight]), rotQuat])
        rotationObject.activate_tool(0)
        R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, liftHeight]), rotQuat])
        R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, liftHeight]), workspaceLoc[1]])

        #update object rotation
        rotationObject.rot += rotAmount

        #If object is correctly rotated, the while loop will end. Otherwise, it continues
    #pickup object again for next move
    R1.set_cartesian([workspaceLoc[0], rotationObject.getApproachQuat(workspaceLoc[1])])
    rotationObject.activate_tool(1)
    R1.set_cartesian([getRelativeVal(workspaceLoc[0], [0, 0, travelHeight]), workspaceLoc[1]])

def main():
    #print("Start")
    #Init robot
    R = initRobot()
    #print("Robot init")
    collector = DatasetCollector(total_photos=picCount, save_path='/Users/arm/Desktop/DataSet')
    #Init Metric Tracker
    metrics = metricTracker(save_path='/Users/arm/Desktop/DataSet', starting_iterations = collector.photo_count)
    #print("Metric Init")
    #print("Collector init")

    #create mobile objects
    cylinder = moveableObj(cylinderStartPos,False, 0, [tGripper, tGripper_adv], tGripperRange, R.set_Do_Grip, defaultSpeed, cylinderRadialEdge, cylinderApproachAngle, cylinderHeight)
    square = moveableObj(squareStartPos,True, 0, [tVac, tVac_adv], tVacRange, R.set_Do_Vacume, squareLiftSpeed, redSquareRadialEdge, squareApproachAngle, squareHeight)

    objectList = [cylinder, square]

    getRelativeJHome(R, [0,0,travelHeight])


    for setCount in range(collector.photo_count, picCount):
        #Main Data Collection Loop - Runs once for every image in the dataset

        #In program data collection and time keeping
        metrics.start_timer()

        #determine object order
        if R.get_joints()[0] >= 0:
            sortReverse = True
        else:
            sortReverse = False
        sorted_objects = sorted(objectList, key=lambda obj: obj.jointLoc[0], reverse=sortReverse)

        
        #for each moving object
        for moveObj in sorted_objects:
            validPosition = False

            while not validPosition:
                ###### DETERMINE NEW RANDOM LOCATION ######
                #Pick random polar coord based on min and max tool values
                randDistance = random_value_with_exclusions(moveObj.toolMinMax[0], moveObj.toolMinMax[1], [], 3)

                #### Pick random polar angle while avoiding potential trouble angles ####

                excludedRanges = []

                #For every other movable object, determine problem angle ranges and add them to list
                for otherObj in objectList:
                    # for every object in the object list, determine all excluded ranges
                    if otherObj != moveObj:
                        excludedRanges.append(get_min_degree_seperation(otherObj))

                randAngle = random_value_with_exclusions(-80.0, 80.0, excludedRanges, 2)

                #convert Polar Coords to X,Y Coords

                randCartesianCoord = polar_to_cartesian(randDistance, randAngle)

                #determine if coord is the minimum distance from all other movable objects
                
                validPosition = True

                #check for conditionals
                if (metrics.run_count % 10 == 0) or (minMoveDistance <= (calculate_distance(randCartesianCoord, [moveObj.cartLoc[0][0], moveObj.cartLoc[0][1]])) <= maxMoveDistance):
                    if (-330 < randCartesianCoord[1] < -270) or (270 < randCartesianCoord[1] < 330):
                        validPosition = False
                    else:
                        for otherObj in objectList:
                            # for every object in the object list
                            if otherObj != moveObj:
                                seperationDistance = calculate_distance(randCartesianCoord, [otherObj.cartLoc[0][0], otherObj.cartLoc[0][1]])
                                if seperationDistance < (minSeperation+otherObj.radialEdge+moveObj.radialEdge):
                                    validPosition = False
                                    break
                else:
                    validPosition = False
            
            #With a valid set of coordinates...

            #set object tool
            R.set_tool(moveObj.tool[0])
            moveObj.activate_tool(0)

            R.set_speed(moveSpeed)
            #Move to Angle
            R.set_joints(getRelativeVal(jHome, [moveObj.polarLoc[1], 0, 0, 0, 0, 0]))

            #move to above objects current location
            
            R.set_cartesian([getAdjustedVal(moveObj.cartLoc[0],[0,0,travelHeight]), moveObj.cartLoc[1]])
            R.set_speed(moveObj.maxLiftSpeed)
            
            #lower
            R.set_cartesian([moveObj.cartLoc[0], moveObj.getApproachQuat(moveObj.cartLoc[1])])
            #grab
            moveObj.activate_tool(1)
            #Raise
            R.set_cartesian([getAdjustedVal(moveObj.cartLoc[0], [0, 0, travelHeight]), moveObj.cartLoc[1]])
            
            #If object has rotation variety, pick a random rotation from 0-359.99
            randRot = 0
            if moveObj.isRot:
                randRot = random_value_with_exclusions(0.0, 90.0, [], 2)
                executeObjectRotation(R, moveObj, randRot)

            
            R.set_speed(moveSpeed)
            #Rotate joint 1 to new position
            R.set_joints(getRelativeVal(jHome, [randAngle, 0, 0, 0, 0, 0]))
            
            #get updated cartesian rotation data
            newQuat = R.get_cartesian()[1]
            #above
            R.set_cartesian([[randCartesianCoord[0],randCartesianCoord[1], travelHeight], newQuat])
            R.set_speed(defaultSpeed)
            
            moveObj.zMod = 0
            #accounting for slanting table
            if randCartesianCoord[1] <= -480:
                    moveObj.zMod = -2
            elif randCartesianCoord[1] >= 320:
                moveObj.zMod = 2

            

            #lower
            R.set_cartesian([[randCartesianCoord[0],randCartesianCoord[1], (moveObj.height + moveObj.zMod)], moveObj.getApproachQuat(newQuat)])

            #Update Object Data Values for: cartLoc, polarLoc, jointLoc, rot

            moveObj.cartLoc = [[randCartesianCoord[0],randCartesianCoord[1], (moveObj.height + moveObj.zMod)], newQuat]
            moveObj.polarLoc = [randDistance, randAngle]
            moveObj.jointLoc = R.get_joints()
            moveObj.rot = randRot

            #drop
            moveObj.activate_tool(0)
            # Raise
            R.set_cartesian([getAdjustedVal(moveObj.cartLoc[0], [0, 0, travelHeight]), moveObj.cartLoc[1]])

        ##All Move Objects have new positions##

        #Move Robot out of the way
        R.set_speed(moveSpeed)
        if R.get_joints()[0] >= 0:
            R.set_joints(jOffScreen)
        else:
            R.set_joints(jOffScreenNeg)

        #Record Data
        photo_name = collector.capture_photo()
        collector.update_csv(photo_name, objectList)

        #Lap Data Collection
        metrics.end_timer()
        metrics.output_to_csv()

        if (metrics.run_count % 15 == 0):
            time.sleep(40)

        print(f"==== {collector.photo_count} / {collector.total_photos} ====")
        print(f"Streak  : {metrics.run_count}")
        print(f"Average Time  : {metrics.get_average_time()}")
        print(f"Previous Loop : {metrics.loop_time}")
        print(f"Estimated Remaining Time: {metrics.format_time(metrics.estimate_remaining_time(picCount))}")
        #Continue to next image...

    R.set_joints(jZero)
    R.close()
    collector.close_class()
    


main()

